# Mapping Old Babylonian readings to Unicode

## Task

We want to map *readings* and *graphemes* in cuneiform corpora to cuneiform unicode characters,
based on extant mapping tables.

We generate a plain mapping that can be used readily by programs that convert from ATF to TF or something else.

## Problem

There are multiple mapping tables, there are several ways to transliterate readings.

## Sources

We take the ATF transliterations from CDLI, for tablets found by a search on AbB and Old Babylonian.

We take the file
[GeneratedSignList.json](https://github.com/Nino-cunei/oldbabylonian/blob/master/sources/writing/GeneratedSignList.json)
with mappings like

```json
        "BANIA": {
            "signName": "BANIA",
            "signNumber": 551,
            "signCunei": "𒑔",
            "codePoint": "",
            "values":
			[
                "BANIA", "AŠ2.UoverU", "5SŪTU"
            ]
        },
        "MA": {
            "signName": "MA",
            "signNumber": 552,
            "signCunei": "𒈠",
            "codePoint": "",
            "values":
			[
                "MA", "PEŠ3", "PEŠŠE", "WA6"
            ]
        },
```

See [transcription](https://github.com/Nino-cunei/oldbabylonian/blob/master/docs/transcription.md)
about the provenance of this file.

# Status

This is work in progress. 
The mapping is needed in the conversion from ATF to TF in the program
[tfFromATF.py](tfFromATF.py).

# Authors

Cale Johnson, Martijn Kokken, Dirk Roorda

# Acknowledgements

We are indebted to **Auday Hussein** for helpfully sending *GeneratedSignList.json* file to us;
to **Alba de Ridder** for hints and comments.

In [1]:
import os
import collections
import re
import json
from unicodedata import name as uname

from tf.app import use

In [2]:
A = use('oldbabylonian', hoist=globals(), lgc=True)

Using TF app oldbabylonian in /Users/dirk/github/annotation/app-oldbabylonian/code
Using Nino-cunei/oldbabylonian/tf - 1.0.2 in /Users/dirk/github


# Local topography

In [10]:
BASE = os.path.expanduser('~/github')
ORG = 'Nino-cunei'
REPO = 'oldbabylonian'

REPO_DIR = f'{BASE}/{ORG}/{REPO}'

WRITING_DIR = f'{REPO_DIR}/sources/writing'

SIGN_FILE = 'GeneratedSignList.json'
SIGN_PATH = f'{WRITING_DIR}/{SIGN_FILE}'

MAPPING_FILE = f'{os.path.abspath("..")}/characters/mapping.tsv'

# Reading collection

We use TF to collect all readings from the corpus in a set.

In [53]:
SRC_FIXES = dict(
  bub='dub',
  umi='um i',
  ura='ra',
  szii='szi',
)

In [54]:
READABLE_TYPES = {'reading', 'grapheme', 'numeral', 'complex'}

tokens = set()

for s in F.otype.s('sign'):
  typ = F.type.v(s)
  if typ not in READABLE_TYPES:
    continue
  reading = F.reading.v(s)
  if reading in SRC_FIXES:
    readings = SRC_FIXES[reading].split()
    for r in readings:
      tokens.add(r)
    continue
  if typ == 'numeral':
    repeat = F.repeat.v(s)
    fraction = F.fraction.v(s)
    if repeat:
      if repeat > 0:
        tokens.add((repeat, reading))
      else:
        tokens.add(reading)
    else:
      tokens.add((fraction, reading))
    continue
  for token in (F.reading.v(s), F.grapheme.v(s)):
    if token:
      tokens.add(token)

print(f'{len(tokens)} different tokens in corpus')

967 different tokens in corpus


# Unicode style versus ATF style

We use mappings between Unicode style transliterations and ATF.

In [55]:
transAscii = {
    'š': 'sz',
    'ṣ': 's,',
    'ś': "s'",
    'ṭ': 't,',
    'ḫ': 'h,',
}

transAscii.update({k.upper(): v.upper() for (k, v) in transAscii.items()})

def makeAscii(r):
  for (rin, rout) in transAscii.items():
    r = r.replace(rin, rout)
  return r

In [56]:
transAscii

{'š': 'sz',
 'ṣ': 's,',
 'ś': "s'",
 'ṭ': 't,',
 'ḫ': 'h,',
 'Š': 'SZ',
 'Ṣ': 'S,',
 'Ś': "S'",
 'Ṭ': 'T,',
 'Ḫ': 'H,'}

In [57]:
REPEAT_INV = dict(
  one=1,
  two=2,
  three=3,
  four=4,
  five=5,
  six=6,
  seven=7,
  eight=8,
  nine=9,
)

REPEAT = {v: k for (k, v) in REPEAT_INV.items()}

In [58]:
FRACTION = {
  '1/2': 'one half',
  '1/3': 'one third',
  '2/3': 'two thirds',
  '1/4': 'one quarter',
  '1/6': 'one sixth',
  '5/6': 'five sixths',
  '1/8': 'one eighth',
}

# Read the sign list

We read the json file with generated signs.

For each sign, we find a list of *values*.

These values correspond to possible readings or graphemes, in short, *tokens*. 
They are in unicode transliteration style.

In the mapping we create, we convert them to plain ATF,
which makes it easier to look them up from our Old Babylonian corpus.

In [59]:
with open(SIGN_PATH) as fh:
  signs = json.load(fh)['signs']

print(f'{len(signs)} signs in the json file')

mapping = collections.defaultdict(set)

for (sign, signData) in signs.items():
  uniStr = signData['signCunei']
  values = signData['values']
  for value in values:
    valueAscii = makeAscii(value)
    mapping[valueAscii].add(uniStr)

print(f'{len(mapping)} distinct values in table')

1768 signs in the json file
8765 distinct values in table


# Token lookup

We look up each Old Babylonian token in the mapping just constructed.

Depending on whether we find 0, 1 or multiple values, we store them in dictionaries
`unmapped`, `unique`, `multiple`.

In [60]:
MAPPING_FIXES = {
    'd': 'dingir',
}

unmapped = set()
unique = {}
multiple = {}

for t in tokens:
  if type(t) is tuple:
    unmapped.add(t)
    continue
  tLookup = MAPPING_FIXES.get(t, t)
  tU = tLookup.upper()
  if tU not in mapping:
    unmapped.add(t)
    continue
  targets = mapping[tU]
  if len(targets) == 1:
    unique[t] = list(targets)[0]
  else:
    multiple[t] = targets
    
print(f'{len(unmapped):>3} unmapped tokens')
print(f'{len(multiple):>3} ambiguously mapped tokens')
print(f'{len(unique):>3} uniquely mapped tokens')

150 unmapped tokens
 50 ambiguously mapped tokens
767 uniquely mapped tokens


# Unmapped tokens

In [61]:
unkey = lambda x: (x[1].lower(), str(x[0])) if type(x) is tuple else (x.lower(), '')

print(f'{len(unmapped):>3} unmapped tokens')
sorted(unmapped, key=unkey)

150 unmapped tokens


["'i",
 'ah',
 'AH',
 'alamusz',
 'asal2',
 (1, 'asz'),
 (2, 'asz'),
 (3, 'asz'),
 (4, 'asz'),
 (5, 'asz'),
 (6, 'asz'),
 (7, 'asz'),
 (8, 'asz'),
 (9, 'asz'),
 'babila2',
 (1, 'ban2'),
 (2, 'ban2'),
 (3, 'ban2'),
 (4, 'ban2'),
 (5, 'ban2'),
 'barig',
 (1, 'barig'),
 (2, 'barig'),
 (3, 'barig'),
 (4, 'barig'),
 (5, 'barig'),
 (1, "bur'u"),
 (2, "bur'u"),
 (3, "bur'u"),
 (4, "bur'u"),
 (5, "bur'u"),
 (1, 'bur3'),
 (2, 'bur3'),
 (3, 'bur3'),
 (4, 'bur3'),
 (5, 'bur3'),
 (6, 'bur3'),
 (8, 'bur3'),
 (9, 'bur3'),
 'dah',
 (1, 'disz'),
 ('1/2', 'disz'),
 ('1/3', 'disz'),
 (13, 'disz'),
 (2, 'disz'),
 ('2/3', 'disz'),
 (3, 'disz'),
 (4, 'disz'),
 (5, 'disz'),
 ('5/6', 'disz'),
 (6, 'disz'),
 (7, 'disz'),
 (8, 'disz'),
 (9, 'disz'),
 'duh',
 'EH',
 'eh',
 'eri11',
 (1, 'esze3'),
 (2, 'esze3'),
 (3, 'esze3'),
 (1, 'gesz'),
 (9, 'gesz'),
 (1, "gesz'u"),
 (2, "gesz'u"),
 (3, "gesz'u"),
 (4, "gesz'u"),
 (7, "gesz'u"),
 (1, 'gesz2'),
 (2, 'gesz2'),
 (3, 'gesz2'),
 (4, 'gesz2'),
 (5, 'gesz2'),
 (6, 

# Fix the unmapped tokens

We look up the unmapped tokens in the unicode table.

In [62]:
cuneiBlocks = {
  'Cuneiform': ('12000', '123FF'),
  'Cuneiform Numbers and Punctuation': ('12400', '1247F'),
  'Early Dynastic Cuneiform': ('12480', '1254F'),
}

In [63]:
cunicode = {}

for (block, (start, end)) in cuneiBlocks.items():
  for u in range(int(start, 16), int(end, 16) + 1):
    c = chr(u)
    name = uname(c, None)
    if name is None:
      continue
    cunicode[name] = c

In [64]:
mapAddition = {}
notFixed = set()

def getLookup(r):
  return (
    r.
    replace("'", '').
    upper().
    replace("SZ", 'SH').
    replace('.', ' TIMES ')
  )
  
  
for t in sorted(unmapped, key=unkey):
  if type(t) is tuple:
    if type(t[0]) is int:
      (repeat, r) = t
      tRepeat = REPEAT.get(repeat, None)
      if tRepeat is None:
        notFixed.add(t)
        continue
      tLookup =  getLookup(r)
      name = f'CUNEIFORM NUMERIC SIGN {tRepeat.upper()} {tLookup}'
      c = cunicode.get(name, None)
      if c is not None:
        mapAddition[t] = c
        continue
      name = f'CUNEIFORM SIGN {tLookup}'
    else:
      (fraction, r) = t
      tFraction = FRACTION.get(fraction, None)
      if tFraction is None:
        notFixed.add(t)
        continue
      tLookup =  getLookup(r)
      name = f'CUNEIFORM NUMERIC SIGN {tFraction.upper()} {tLookup}'
  else:
    tLookup =  getLookup(t)
    name = f'CUNEIFORM SIGN {tLookup}'
  c = cunicode.get(name, None)
  if c is None:
    notFixed.add(t)
  else:
    mapAddition[t] = c

print(f'fixed {len(mapAddition)} out of {len(unmapped)}')

if mapAddition:
  print('FIXED')
  for (t, c) in sorted(mapAddition.items(), key=unkey):
    print(f'\t{str(t):<15} => {c}')
else:
  print('NOTHING FIXED')
  
if notFixed:
  print('UNFIXED')
  for t in sorted(notFixed, key=unkey):
    print(f'\t{str(t):<15} => ?')
else:
  print('ALL FIXED')

fixed 67 out of 150
FIXED
	asal2           => 𒀷
	(1, 'asz')      => 𒀸
	(1, 'disz')     => 𒁹
	(2, 'disz')     => 𒁹
	duh             => 𒂃
	(2, 'gisz')     => 𒄑
	HA              => 𒄩
	ha              => 𒄩
	hal             => 𒄬
	HI              => 𒄭
	hi              => 𒄭
	HU              => 𒄷
	hu              => 𒄷
	hub2            => 𒄸
	'i              => 𒄿
	mah             => 𒈤
	pesz2           => 𒉾
	(1, 'szar2')    => 𒊹
	(1, 'u')        => 𒌋
	(2, 'u')        => 𒌋
	(3, 'u')        => 𒌋
	(2, 'asz')      => 𒐀
	(3, 'asz')      => 𒐁
	(4, 'asz')      => 𒐂
	(5, 'asz')      => 𒐃
	(6, 'asz')      => 𒐄
	(7, 'asz')      => 𒐅
	(8, 'asz')      => 𒐆
	(9, 'asz')      => 𒐇
	(3, 'disz')     => 𒐈
	(4, 'disz')     => 𒐉
	(5, 'disz')     => 𒐊
	(6, 'disz')     => 𒐋
	(7, 'disz')     => 𒐌
	(8, 'disz')     => 𒐍
	(9, 'disz')     => 𒐎
	(4, 'u')        => 𒐏
	(5, 'u')        => 𒐐
	(1, 'gesz2')    => 𒐕
	(2, 'gesz2')    => 𒐖
	(3, 'gesz2')    => 𒐗
	(4, 'gesz2')    => 𒐘
	(5, 'gesz2')    => 𒐙
	(6, 'gesz2')    => 𒐚
	(7, '

# Solutions

Most of the remaining problems above got solved by a 
[table provided by Martijn Kokken](https://github.com/Nino-cunei/oldbabylonian/blob/master/sources/writing/MartijnKokken.txt)

In [85]:
MAPPING_SOLUTIONS = dict(
  ah=('HIxNUN', 'U12134'),
  AH=('HIxNUN', 'U12134'),
  alamusz=('TAxHI', 'U122ED'),
  babila2=('KA2.AN.RA', 'U1218D U1202D U1228F'),
  dah=('MU/MU', 'U1222D'),
  eh=('HIxNUN', 'U12134'),
  EH=('HIxNUN', 'U12134'),
  eri11=('AB gunû', 'U12015'),
  geszimmar=('ŠA6', 'U122B7'),
  gudu4=('HIxNUN.ME', 'U12134 U12228'),
  had2=('UD', 'U12313'),
  har=('HIxAŠ2', 'U1212F'),
  HAR=('HIxAŠ2', 'U1212F'),
  he=('HI', 'U1212D'),
  he2=('GAN', 'U120F6'),
  hun=('EŠ2', 'U120A0'),
  hur=('HIxAŠ2', 'U1212F'),
  huz=('LUM', 'U1221D'),
  ih=('HIxNUN', 'U12134'),
  IH=('HIxNUN', 'U12134'),
  itu=('UDxU.U.U', 'U12317'),
  KA=('KA TA', 'U12157 U122EB'),
  kislah=('KI.UD', 'U121A0 U12313'),
  lah=('UD', 'U12313'),
  lah4=('DU / DU', 'U1207B'),
  lah5=('DU.DU', 'U1207A U1207A'),
  lah6=('DU', 'U1207A'),
  lal3=('TAxHI', 'U122ED'),
  muhaldim=('MU', 'U1222C'),
  nigar=('U.UD.KID', 'U1230B U12313 U121A4'),
  nirah=('MUŠ', 'U12232'),
  sa10=('NINDA2xŠE', 'U1225A'),
  sahar=('IŠ', 'U12156'),
  siskur2=('AMARxŠE.AMARxŠE', 'U1202C U1202C'),
  szagina=('GIR3.ARAD', 'U1210A U12034'),
  szah=('ŠUBUR', 'U122DA'),
  szah2=('DUN', 'U12084'),
  szandana=('GAL.NI', 'U120F2 U1224C'),
  tah=('MU/MU', 'U1222D'),
  tap=('TAB', 'U122F0'),
  udru=('AŠ2', 'U1203E'),
  UH=('HIxNUN', 'U12134'),
  uh=('HIxNUN', 'U12134'),
  UH2=('UD.KUŠU2', 'U12313 U121B5'),
  uh2=('UD.KUŠU2', 'U12313 U121B5'),
  uh3=('KUŠU2', 'U121B5'),
  UH3=('KUŠU2', 'U121B5'),
  ukken=('URUxBAR', 'U1233A'),
  unu=('AB gunû', 'U12015'),
)
MAPPING_SOLUTIONS.update({
  'barig': ('', 'U12079'),
  '1(barig)': ('', 'U12079'),
  '2(barig)': ('', 'U12079 U12079'),
  '3(barig)': ('', 'U12079 U12079 U12079'),
  '4(barig)': ('', 'U1235D'),
  '5(barig)': ('', 'U12125'),
  'bur3': ('', 'U1230B'),
  "bur'u": ('', 'U12434'),
  '1(bur3)': ('', 'U1230B'),
  '2(bur3)': ('', 'U1230B U1230B'),
  '3(bur3)': ('', 'U1230B U1230B U1230B'),
  '4(bur3)': ('', 'U1240F'),
  '5(bur3)': ('', 'U12410'),
  '6(bur3)': ('', 'U12411'),
  '7(bur3)': ('', 'U12412'),
  '8(bur3)': ('', 'U12413'),
  '9(bur3)': ('', 'U12414'),
  '1/2(disz)': ('', 'U12226'),
  '13(disz)': ('', 'U12399 U12408'),
  'gesz2': ('', 'U12415'),
  "gesz'u": ('', 'U1241E'),
  'szar2': ('', 'U122B9'),
})

MAPPING_SOLUTIONSX = {}

for (token, (grapheme, uniChars)) in MAPPING_SOLUTIONS.items():
  uniStr = ''.join(chr(int(uc[1:], 16)) for uc in uniChars.split())
  MAPPING_SOLUTIONSX[token] = uniStr
MAPPING_SOLUTIONSX

{'ah': '𒄴',
 'AH': '𒄴',
 'alamusz': '𒋭',
 'babila2': '𒆍𒀭𒊏',
 'dah': '𒈭',
 'eh': '𒄴',
 'EH': '𒄴',
 'eri11': '𒀕',
 'geszimmar': '𒊷',
 'gudu4': '𒄴𒈨',
 'had2': '𒌓',
 'har': '𒄯',
 'HAR': '𒄯',
 'he': '𒄭',
 'he2': '𒃶',
 'hun': '𒂠',
 'hur': '𒄯',
 'huz': '𒈝',
 'ih': '𒄴',
 'IH': '𒄴',
 'itu': '𒌗',
 'KA': '𒅗𒋫',
 'kislah': '𒆠𒌓',
 'lah': '𒌓',
 'lah4': '𒁻',
 'lah5': '𒁺𒁺',
 'lah6': '𒁺',
 'lal3': '𒋭',
 'muhaldim': '𒈬',
 'nigar': '𒌋𒌓𒆤',
 'nirah': '𒈲',
 'sa10': '𒉚',
 'sahar': '𒅖',
 'siskur2': '𒀬𒀬',
 'szagina': '𒄊𒀴',
 'szah': '𒋚',
 'szah2': '𒂄',
 'szandana': '𒃲𒉌',
 'tah': '𒈭',
 'tap': '𒋰',
 'udru': '𒀾',
 'UH': '𒄴',
 'uh': '𒄴',
 'UH2': '𒌓𒆵',
 'uh2': '𒌓𒆵',
 'uh3': '𒆵',
 'UH3': '𒆵',
 'ukken': '𒌺',
 'unu': '𒀕',
 'barig': '𒁹',
 '1(barig)': '𒁹',
 '2(barig)': '𒁹𒁹',
 '3(barig)': '𒁹𒁹𒁹',
 '4(barig)': '𒍝',
 '5(barig)': '𒄥',
 'bur3': '𒌋',
 "bur'u": '𒐴',
 '1(bur3)': '𒌋',
 '2(bur3)': '𒌋𒌋',
 '3(bur3)': '𒌋𒌋𒌋',
 '4(bur3)': '𒐏',
 '5(bur3)': '𒐐',
 '6(bur3)': '𒐑',
 '7(bur3)': '𒐒',
 '8(bur3)': '𒐓',
 '9(bur3)': '𒐔',
 '1/2(disz

# Ambiguously mapped readings

In [86]:
print(f'{len(multiple):>3} ambiguously mapped readings')
for r in sorted(multiple):
  unis = multiple[r]
  uniStr = ' - '.join(sorted(unis))
  print(f'{r} => ({len(unis)}) => {uniStr}')

 50 ambiguously mapped readings
IA => (2) => 𒅀 - 𒉿
IL => (2) => 𒀧 - 𒅋
IRI => (2) => 𒅕 - 𒌷
KAM => (2) => 𒄭𒁁 - 𒄰
LUM => (2) => 𒈝 - 𒋞
USZ => (2) => 𒍑 - 𒍖
UZ => (2) => 𒊻 - 𒍖
WA => (2) => 𒁀 - 𒉿
ba4 => (3) => 𒀀𒀭𒂷 - 𒂷 - 𒍝𒂷𒂷
ba6 => (2) => 𒁀𒌑 - 𒌑
bara2 => (2) => 𒁁 - 𒁈
bum => (2) => 𒅤 - 𒆃
buru14 => (2) => 𒂘 - 𒂙
dabin => (2) => 𒂠𒊺 - 𒍥𒊺
dilmun => (3) => 𒉌𒌇 - 𒊩𒄸 - 𒊩𒌇
eri => (2) => 𒅕 - 𒌷
erisz => (2) => 𒊩𒈠 - 𒊩𒌆
gala => (3) => 𒃲 - 𒍑𒆪 - 𒍓
gin7 => (2) => 𒁶 - 𒄀
gurusz => (2) => 𒄨 - 𒆗
ia => (2) => 𒅀 - 𒉿
idim => (2) => 𒁁 - 𒅂
ii => (2) => 𒅀 - 𒉿
il => (2) => 𒀧 - 𒅋
iri => (2) => 𒅕 - 𒌷
isz8 => (2) => 𒀹 - 𒌋
iu => (2) => 𒅀 - 𒉿
kam => (2) => 𒄭𒁁 - 𒄰
kesz2 => (2) => 𒂡 - 𒆟
kesz3 => (2) => 𒋙𒀭𒄲 - 𒋙𒀭𒄲𒆠
lum => (2) => 𒈝 - 𒋞
munu4 => (2) => 𒉽𒉽 - 𒉽𒊺𒉽
ne3 => (2) => 𒄊 - 𒊊
nergal => (2) => 𒄊𒀕𒃲 - 𒊊𒀕𒃲
pa2 => (2) => 𒁀 - 𒁀𒌑
pirig => (2) => 𒄊 - 𒊊
puzur4 => (2) => 𒅤𒊭 - 𒆃𒊭
sig17 => (2) => 𒄀 - 𒆬
sze20 => (2) => 𒂠 - 𒅆
t,a2 => (2) => 𒋫 - 𒋬
til => (2) => 𒁁 - 𒌀
us => (2) => 𒊻 - 𒍖
usa => (2) => 𒐍 - 𒑄
usz => (2) => 𒍑 - 𒍖
usz2 => (2) => 𒁁 

# Uniquely mapped readings

In [87]:
print(f'{len(unique):>3} uniquely mapped readings')
for r in sorted(unique):
  print(f'{r:>10} => {unique[r]}')

767 uniquely mapped readings
         A => 𒀀
        AB => 𒀊
        AD => 𒀜
        AG => 𒀝
        AK => 𒀝
        AL => 𒀠
        AM => 𒄠
        AN => 𒀭
        AR => 𒅈
      ARAD => 𒀴
     ARAD2 => 𒀵
       AS, => 𒊍
       AS2 => 𒀾
       ASZ => 𒀸
        AZ => 𒊍
        BA => 𒁀
       BAD => 𒁁
       BAR => 𒁇
        BE => 𒁁
        BI => 𒁉
        BU => 𒁍
       BUR => 𒁓
        DA => 𒁕
       DAM => 𒁮
        DI => 𒁲
       DIM => 𒁴
       DIN => 𒁷
      DISZ => 𒁹
        DU => 𒁺
         E => 𒂊
      EDIN => 𒂔
        EK => 𒅅
        EL => 𒂖
        ER => 𒅕
        GA => 𒂵
       GAG => 𒆕
       GAL => 𒃲
      GAN2 => 𒃷
       GAR => 𒃻
       GAZ => 𒄤
      GESZ => 𒄑
        GI => 𒄀
       GIR => 𒄫
      GIR2 => 𒄈
        GU => 𒄖
         I => 𒄿
        IB => 𒅁
        ID => 𒀉
        IG => 𒅅
        IK => 𒅅
       IL2 => 𒅍
        IM => 𒅎
        IN => 𒅔
        IR => 𒅕
       ISZ => 𒅖
        IZ => 𒄑
        KA => 𒅗
       KAB => 𒆏
        KI => 𒆠
       KIB => 𒄒
        KU 

# Write the mapping file

In [88]:
pairs = {}
for (k, vs) in multiple.items():
  pairs[k] = sorted(vs)[0]
for (t, v) in mapAddition.items():
  k = f'{t[0]}({t[1]})' if type(t) is tuple else t
  pairs[k] = v
for (k, v) in MAPPING_SOLUTIONSX.items():
  pairs[k] = v
for (k, v) in unique.items():
  pairs[k] = v

with open(MAPPING_FILE, 'w') as mf:
  for (k,v) in sorted(pairs.items()):
    mf.write(f'{k}\t{v}\n')
print(f'{len(pairs)} entries written to {MAPPING_FILE}')

953 entries written to /Users/dirk/github/Nino-cunei/oldbabylonian/characters/mapping.tsv
